# 🏙️ Urban Growth GPU Pipeline (Simple Version)
This notebook loads the Austin Construction Permits CSV (~2GB), extracts useful insights, 
and prepares a clean dataset for downstream models (RAG, clustering, predictions, etc.).

In [1]:
import cudf
import cupy as cp
import numpy as np
import re
from cudf.core.column import as_column

data_path = "/home/asus/atx-hackathon/data/Issued_Construction_Permits_20251213.csv"
print("Loading file:", data_path)

Loading file: /home/asus/atx-hackathon/data/Issued_Construction_Permits_20251213.csv


## 🔹 Step 1 — Load CSV into cuDF (GPU)

In [5]:
import cudf

dfs = []
chunk_size = 256 * 1024 * 1024  # 256MB
offset = 0

while True:
    try:
        gdf_chunk = cudf.read_csv(
            data_path,
            byte_range=(offset, chunk_size)
        )
        dfs.append(gdf_chunk)
        offset += chunk_size
    except Exception:
        break

df = cudf.concat(dfs)
df
df.head()

,Permit Type,Permit Type Desc,Permit Num,Permit Class Mapped,Permit Class,Work Class,Condominium,Project Name,Description,TCAD ID,...,Contractor Zip,Applicant Full Name,Applicant Organization,Applicant Phone,Applicant Address 1,Applicant Address 2,Applicant City,Applicant Zip,Certificate Of Occupancy,Total Lot SQFT
0,PP,Plumbing Permit,2025-142580 PP,Residential,Residential,Irrigation,<NA>,13109 TAMAR CT,New install,0262201310,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,PP,Plumbing Permit,2025-125697 PP,Residential,Residential,Irrigation,<NA>,9013 HAMADRYAS DR,Irrigation install,0340130503,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,PP,Plumbing Permit,2025-125699 PP,Residential,Residential,Irrigation,<NA>,9015 HAMADRYAS DR,Irrigation install,0340130506,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,PP,Plumbing Permit,2025-133647 PP,Residential,Residential,Irrigation,<NA>,1405 IBERVILLE DR,Install sprinkler system,0274281601,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,PP,Plumbing Permit,2025-125469 PP,Residential,Residential,Irrigation,<NA>,5511 FORKS RD,irrigation install,0438011501,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## 🔹 Step 2 — Basic Cleanup
- Normalize text fields
- Fill missing values
- Create lower‑case searchable versions

In [6]:
text_cols = ["DESCRIPTION", "WORK_CLASS", "ISSUED_DATE"]

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna("")
        df[col + "_clean"] = df[col].str.lower()
        
df.head()

,Permit Type,Permit Type Desc,Permit Num,Permit Class Mapped,Permit Class,Work Class,Condominium,Project Name,Description,TCAD ID,...,Contractor Zip,Applicant Full Name,Applicant Organization,Applicant Phone,Applicant Address 1,Applicant Address 2,Applicant City,Applicant Zip,Certificate Of Occupancy,Total Lot SQFT
0,PP,Plumbing Permit,2025-142580 PP,Residential,Residential,Irrigation,<NA>,13109 TAMAR CT,New install,0262201310,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,PP,Plumbing Permit,2025-125697 PP,Residential,Residential,Irrigation,<NA>,9013 HAMADRYAS DR,Irrigation install,0340130503,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,PP,Plumbing Permit,2025-125699 PP,Residential,Residential,Irrigation,<NA>,9015 HAMADRYAS DR,Irrigation install,0340130506,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,PP,Plumbing Permit,2025-133647 PP,Residential,Residential,Irrigation,<NA>,1405 IBERVILLE DR,Install sprinkler system,0274281601,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,PP,Plumbing Permit,2025-125469 PP,Residential,Residential,Irrigation,<NA>,5511 FORKS RD,irrigation install,0438011501,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## 🔹 Step 3 — Feature Extraction (Simple Patterns)
We create quick flags for:
🏠 ADUs (Accessory Dwelling Units)
🔌 EV Chargers
🏢 Multi‑family projects
☀️ Solar installs
🛠️ Renovations

In [7]:
def make_flag(col, pattern):
    return df[col].str.contains(pattern, regex=True)

df["flag_adu"]   = make_flag("DESCRIPTION_clean", r"\badu\b| accessory dwelling")
df["flag_ev"]    = make_flag("DESCRIPTION_clean", r"ev charger|electric vehicle")
df["flag_multi"] = make_flag("DESCRIPTION_clean", r"multi[- ]family|apt|apartment")
df["flag_solar"] = make_flag("DESCRIPTION_clean", r"solar|pv system")
df["flag_reno"]  = make_flag("DESCRIPTION_clean", r"renovation|remodel|repair")

df[["flag_adu", "flag_ev", "flag_multi", "flag_solar", "flag_reno"]].head()

KeyError: 'DESCRIPTION_clean'

## 🔹 Step 4 — Quick Insights
Counts of major project types.

In [ ]:
counts = {
    "ADUs": int(df.flag_adu.sum()),
    "EV Chargers": int(df.flag_ev.sum()),
    "Multi-Family": int(df.flag_multi.sum()),
    "Solar": int(df.flag_solar.sum()),
    "Renovations": int(df.flag_reno.sum())
}

counts

## 🔹 Step 5 — Save a Clean Parquet File
This output can be used directly by your RAG/chatbot or your ML forecasting pipeline.

In [ ]:
output_path = "/home/asus/atx-hackathon/data/permits_clean.parquet"
df.to_parquet(output_path)
print("Saved:", output_path)

## ✔️ Complete!
You now have:
- GPU-loaded 2M row dataset
- Cleaned text
- Flags for major urban trends
- Summary stats
- Parquet file ready for RAG, analysis, or clustering